In [1]:
from lane_changing import *
from data2 import build_params

def build_result_mat(results_dry, results_wet):
    p_dry = np.zeros([3])
    p_wet = np.zeros([3])

    p_dry[ results_dry['a_opt_0'] ] += results_dry['prob_sensor_00'] + results_dry['prob_sensor_10']
    p_dry[ results_dry['a_opt_1'] ] += results_dry['prob_sensor_10'] + results_dry['prob_sensor_11']

    p_wet[ results_dry['a_opt_0'] ] += results_wet['prob_sensor_00'] + results_wet['prob_sensor_10']
    p_wet[ results_dry['a_opt_1'] ] += results_wet['prob_sensor_10'] + results_wet['prob_sensor_11']

    p_mat = np.array([p_dry, p_wet])   
    
    prob_s_dry = np.round(results_dry['prob_s'], 3)
    #prob_s_dry = prob_s_dry/np.sum(prob_s_dry)
    prob_s_wet = np.round(results_wet['prob_s'], 3)
    #prob_s_wet = prob_s_dry/np.sum(prob_s_wet)
    prob_s_mat = np.array([prob_s_dry, prob_s_wet])
    
    return p_mat, prob_s_mat

# Section 3.2.2

For the reference experiments we choose the following parameterization:

* ADS weights: `wA  = np.array([3/100, 22/100, 0, 3/100, 22/100, 0, 22/100, 31/100])`
* ADS risk aversion coefficient: `rho_A=0.5`

* MV weights: `np.array([0.1, 0.5, 0.05, 0.35])`
* MV risk aversion coefficient: `rho_MV=1.0`

* 1 passenger in ADS, 1 passenger in MV, 1 pedestrian.

Notice that this is a quite risky driving scene, in the sense that both the
ADS and the MV give a lot of weight to speed (last component of the vectors)
compared to the weight given to safety.


In [2]:
wA  = np.array([3/100, 22/100, 0, 3/100, 22/100, 0, 22/100, 31/100])
wMV = np.array([0.1, 0.5, 0.05, 0.35])

params = build_params(theta2=1.0, theta3=1.0, theta4=1.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

In [3]:
results_wet = ads.simulate(theta=1)
results_dry = ads.simulate(theta=0)
p_mat, prob_s_mat = build_result_mat(results_dry, results_wet)

We first show the $p(m|a)$ inferred by the ADS.

In [4]:
df_probs = pd.DataFrame(ads.pam)
df_probs.columns = ['$m_1$', '$m_2$', '$m_3$']
df_probs.index = ['$a_1$', '$a_2$', '$a_3$']
df_probs

,$m_1$,$m_2$,$m_3$
$a_1$,0.0,0.846,0.154
$a_2$,1.0,0.000,0.000
$a_3$,1.0,0.000,0.000


When ADS decides to change lane, the ADS thinks that the MV will most likely deccelerate or (with less probability) change lane.

When ADS decides to remain it is very sure that MV will accelerate.

If ADS decides to make and emergency stop, the ADS is very sure that MV will make one too.

In [6]:
df_probs.style.to_latex()

'\\begin{tabular}{lrrr}\n & $m_1$ & $m_2$ & $m_3$ \\\\\n$a_1$ & 0.000000 & 0.846000 & 0.154000 \\\\\n$a_2$ & 1.000000 & 0.000000 & 0.000000 \\\\\n$a_3$ & 1.000000 & 0.000000 & 0.000000 \\\\\n\\end{tabular}\n'

### Table 4

In [7]:
d = {
     'Pavement': ['Dry', 'Wet'],
     '$a_1$'   : p_mat[:,0],
     '$a_2$'   : p_mat[:,1],
     '$a_3$'   : p_mat[:,2],
    }
df = pd.DataFrame(d).set_index('Pavement')
df

,$a_1$,$a_2$,$a_3$
Pavement,,,
Dry,0.95,0.05,0.0
Wet,0.05,0.95,0.0


When pavement is dry, the ADS decides to change lane more often than not. 
When pavement is wet, it tends to be more conservative and decides more often not to change lane.

In [10]:
df.style.to_latex()

'\\begin{tabular}{lrrr}\n & $a_1$ & $a_2$ & $a_3$ \\\\\nPavement &  &  &  \\\\\nDry & 0.950000 & 0.050000 & 0.000000 \\\\\nWet & 0.050000 & 0.950000 & 0.000000 \\\\\n\\end{tabular}\n'

### Table 5

In [28]:
d = {
     'Pavement': ['Dry', 'Wet'],
     '$s_1$'   : prob_s_mat[:,0],
     '$s_2$'   : prob_s_mat[:,1],
     '$s_3$'   : prob_s_mat[:,2],
     '$s_4$'   : prob_s_mat[:,3],
     '$s_5$'   : prob_s_mat[:,4],
    

    }
df = pd.DataFrame(d).set_index('Pavement')
df

,$s_1$,$s_2$,$s_3$,$s_4$,$s_5$
Pavement,,,,,
Dry,0.158,0.158,0.681,0.0,0.003
Wet,0.017,0.017,0.682,0.0,0.285


This is a consequence of the actions chosen. When pavement is dry, ADS chooses to change lane more often and thus, risky scenarios ($s_1$ and $s_2$) occurr more often. 

In [30]:
df.style.to_latex()

'\\begin{tabular}{lrrrrr}\n & $s_1$ & $s_2$ & $s_3$ & $s_4$ & $s_5$ \\\\\nPavement &  &  &  &  &  \\\\\nDry & 0.158000 & 0.158000 & 0.681000 & 0.000000 & 0.003000 \\\\\nWet & 0.017000 & 0.017000 & 0.682000 & 0.000000 & 0.285000 \\\\\n\\end{tabular}\n'

### Table 6

Same weights as before. We compare 3 configurations.

* No passengers in ADS, 5 passengers in MV, 1 pedestrian.
* 4 in ADS, 1 passengers in MV, 1 pedestrian.
* 5 in ADS, 5 passengers in MV, 0 pedestrians.

In [31]:
wA  = np.array([3/100, 22/100, 0, 3/100, 22/100, 0, 22/100, 31/100])
wMV = np.array([0.1, 0.5, 0.05, 0.35])


In [32]:
params = build_params(theta2=0.0, theta3=5.0, theta4=1.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

results_wet_1 = ads.simulate(theta=1)
results_dry_1 = ads.simulate(theta=0)

p_mat_1, prob_s_mat_1 = build_result_mat(results_dry_1, results_wet_1)

In [33]:
params = build_params(theta2=4.0, theta3=1.0, theta4=1.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

results_wet_2 = ads.simulate(theta=1)
results_dry_2 = ads.simulate(theta=0)

p_mat_2, prob_s_mat_2 = build_result_mat(results_dry_2, results_wet_2)

In [34]:
params = build_params(theta2=5.0, theta3=5.0, theta4=0.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

results_wet_3 = ads.simulate(theta=1)
results_dry_3 = ads.simulate(theta=0)

p_mat_3, prob_s_mat_3 = build_result_mat(results_dry_3, results_wet_3)

In [35]:
tuples =  [('1', 'Dry'),('1', 'Wet'),('2', 'Dry'),('3', 'Wet'),('3', 'Dry'),('3', 'Wet')]
index = pd.MultiIndex.from_tuples(tuples, names=["Configuration", "Pavement"])

conf1 = np.concatenate([p_mat_1, prob_s_mat_1], axis=1)
conf2 = np.concatenate([p_mat_2, prob_s_mat_2], axis=1)
conf3 = np.concatenate([p_mat_3, prob_s_mat_3], axis=1)
tab6  = pd.DataFrame( np.concatenate([conf1, conf2, conf3], axis=0) )
tab6.columns = ['$a_1$', '$a_2$', '$a_3$', '$s_1$', '$s_2$', '$s_3$', '$s_4$', '$s_5$']
tab6.index   = index
tab6

$a_1$  $a_2$  $a_3$  $s_1$  $s_2$  $s_3$  $s_4$  $s_5$
Configuration Pavement                                                        
1             Dry        0.95   0.05    0.0  0.040  0.040  0.918    0.0  0.003
              Wet        0.05   0.95    0.0  0.017  0.017  0.682    0.0  0.285
2             Dry        0.00   1.00    0.0  0.000  0.000  0.950    0.0  0.050
3             Wet        0.00   1.00    0.0  0.000  0.000  0.700    0.0  0.300
              Dry        0.00   1.00    0.0  0.000  0.000  0.950    0.0  0.050
              Wet        0.00   1.00    0.0  0.000  0.000  0.700    0.0  0.300

In configuration 1-Dry, as in the previous case,
the ADS decides to change lane 95% of the times.
However, in this case, the probability of risky scenarios $s_1$ and $s_2$ is much lower.
This is a consequence of the fact that, as the MV has more passengers, it tends to be more autoprotective, and instead of deciding to accelerate most of the time, it decides to change lane.

For configuration 2 the MV has less passengers so tends to be less protective. Given that the APS cares equally about its passengers and the MV ones, this induces the APS being more consevatives and thus choosing to remain in the lane. 

In configuration 3, given the high number of people at risk, the ADS decides to remain in the lane too.


In [19]:
tab6.style.to_latex()

'\\begin{tabular}{llrrrrrrrr}\n &  & $a_1$ & $a_2$ & $a_3$ & $s_1$ & $s_2$ & $s_3$ & $s_4$ & $s_5$ \\\\\nConfiguration & Pavement &  &  &  &  &  &  &  &  \\\\\n\\multirow[c]{2}{*}{1} & Dry & 0.950000 & 0.050000 & 0.000000 & 0.040000 & 0.040000 & 0.918000 & 0.000000 & 0.003000 \\\\\n & Wet & 0.050000 & 0.950000 & 0.000000 & 0.017000 & 0.017000 & 0.682000 & 0.000000 & 0.285000 \\\\\n2 & Dry & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.950000 & 0.000000 & 0.050000 \\\\\n\\multirow[c]{3}{*}{3} & Wet & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.700000 & 0.000000 & 0.300000 \\\\\n & Dry & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.950000 & 0.000000 & 0.050000 \\\\\n & Wet & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.700000 & 0.000000 & 0.300000 \\\\\n\\end{tabular}\n'

## Impact of Number of Pedestrians

With the previous weight configuration, the number of pedestrians in the scene did not affect the ADS action because speed was very important (compared to safety). As a consequence, making an emergency stop was very expensive and never optimal.

We define now an ADS in which the weight given to speed is much less, and see how the number of pedestrians affect the optimal actions taken.
For this case, both the ADS and MV have 5 passengers.

In [46]:
wA  = np.array([5/100, 32/100, 0, 5/100, 32/100, 0, 22/100, 7/100])
wMV = np.array([0.05, 0.1, 0.05, 0.8])

params = build_params(theta2=5.0, theta3=0.0, theta4=0.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)
results_wet_1 = ads.simulate(theta=1)
results_dry_1 = ads.simulate(theta=0)
p_mat_1, prob_s_mat_1 = build_result_mat(results_dry_1, results_wet_1)

In [47]:
params = build_params(theta2=5.0, theta3=5.0, theta4=5.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)
results_wet_2 = ads.simulate(theta=1)
results_dry_2 = ads.simulate(theta=0)
p_mat_2, prob_s_mat_2 = build_result_mat(results_dry_2, results_wet_2)

In [48]:
tuples =  [('1', 'Dry'),('1', 'Wet'),('2', 'Dry'), ('2', 'Wet')]
index = pd.MultiIndex.from_tuples(tuples, names=["Configuration", "Pavement"])

conf1 = np.concatenate([p_mat_1, prob_s_mat_1], axis=1)
conf2 = np.concatenate([p_mat_2, prob_s_mat_2], axis=1)
tab7  = pd.DataFrame( np.concatenate([conf1, conf2], axis=0) )
tab7.columns = ['$a_1$', '$a_2$', '$a_3$', '$s_1$', '$s_2$', '$s_3$', '$s_4$', '$s_5$']
tab7.index   = index
tab7

$a_1$  $a_2$  $a_3$  $s_1$  $s_2$  $s_3$  $s_4$  $s_5$
Configuration Pavement                                                        
1             Dry         0.0   0.95   0.05    0.0    0.0  0.902   0.05  0.048
              Wet         0.0   0.05   0.95    0.0    0.0  0.035   0.95  0.015
2             Dry         0.0   1.00   0.00    0.0    0.0  0.950   0.00  0.050
              Wet         0.0   1.00   0.00    0.0    0.0  0.700   0.00  0.300

In configuration 1, there are no pedestrians in the road and, given that both the ADS and the MV are highly populated, the ADS chooses to make an emergency stop, as this won't put anybody at risk. However, when there are 5 pedestrias (configuration 2) the ADS does not choose the emergency stop anymore.

In [42]:
tab7.style.to_latex()

'\\begin{tabular}{llrrrrrrrr}\n &  & $a_1$ & $a_2$ & $a_3$ & $s_1$ & $s_2$ & $s_3$ & $s_4$ & $s_5$ \\\\\nConfiguration & Pavement &  &  &  &  &  &  &  &  \\\\\n\\multirow[c]{2}{*}{1} & Dry & 0.000000 & 0.950000 & 0.050000 & 0.000000 & 0.000000 & 0.902000 & 0.050000 & 0.048000 \\\\\n & Wet & 0.000000 & 0.050000 & 0.950000 & 0.000000 & 0.000000 & 0.035000 & 0.950000 & 0.015000 \\\\\n\\multirow[c]{2}{*}{2} & Dry & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.950000 & 0.000000 & 0.050000 \\\\\n & Wet & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.700000 & 0.000000 & 0.300000 \\\\\n\\end{tabular}\n'

## Impact of Risk aversion Coefficient

We recover the initial weight configuration and study the effect of the ADS risk aversion coefficient.
We compare $\rho = 0.5$ and $\rho = 2.5$.

In [50]:
wA  = np.array([3/100, 22/100, 0, 3/100, 22/100, 0, 22/100, 31/100])
wMV = np.array([0.1, 0.5, 0.05, 0.35])

params = build_params(theta2=1.0, theta3=1.0, theta4=1.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)
results_wet_1 = ads.simulate(theta=1)
results_dry_1 = ads.simulate(theta=0)
p_mat_1, prob_s_mat_1 = build_result_mat(results_dry_1, results_wet_1)

In [51]:
params = build_params(theta2=1.0, theta3=1.0, theta4=1.0, 
                      rho_A=2.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)
results_wet_2 = ads.simulate(theta=1)
results_dry_2 = ads.simulate(theta=0)
p_mat_2, prob_s_mat_2 = build_result_mat(results_dry_2, results_wet_2)

In [52]:
tuples =  [('$\rho=0.5$', 'Dry'),('$\rho=0.5$', 'Wet'),('$\rho=2.5$', 'Dry'), ('$\rho=2.5$', 'Wet')]
index = pd.MultiIndex.from_tuples(tuples, names=["Configuration", "Pavement"])

conf1 = np.concatenate([p_mat_1, prob_s_mat_1], axis=1)
conf2 = np.concatenate([p_mat_2, prob_s_mat_2], axis=1)
tab8  = pd.DataFrame( np.concatenate([conf1, conf2], axis=0) )
tab8.columns = ['$a_1$', '$a_2$', '$a_3$', '$s_1$', '$s_2$', '$s_3$', '$s_4$', '$s_5$']
tab8.index   = index
tab8

$a_1$  $a_2$  $a_3$  $s_1$  $s_2$  $s_3$  $s_4$  $s_5$
Configuration Pavement                                                        
$\rho=0.5$    Dry        0.95   0.05    0.0  0.158  0.158  0.681    0.0  0.003
              Wet        0.05   0.95    0.0  0.017  0.017  0.682    0.0  0.285
$\rho=2.5$    Dry        0.00   1.00    0.0  0.000  0.000  0.950    0.0  0.050
              Wet        0.00   1.00    0.0  0.000  0.000  0.700    0.0  0.300

As expected, when the risk aversion coefficient is high, the ADS tends to be more conservative and chooses to remain in the lane.

In [53]:
tab8.style.to_latex()

'\\begin{tabular}{llrrrrrrrr}\n &  & $a_1$ & $a_2$ & $a_3$ & $s_1$ & $s_2$ & $s_3$ & $s_4$ & $s_5$ \\\\\nConfiguration & Pavement &  &  &  &  &  &  &  &  \\\\\n\\multirow[c]{2}{*}{$\rho=0.5$} & Dry & 0.950000 & 0.050000 & 0.000000 & 0.158000 & 0.158000 & 0.681000 & 0.000000 & 0.003000 \\\\\n & Wet & 0.050000 & 0.950000 & 0.000000 & 0.017000 & 0.017000 & 0.682000 & 0.000000 & 0.285000 \\\\\n\\multirow[c]{2}{*}{$\rho=2.5$} & Dry & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.950000 & 0.000000 & 0.050000 \\\\\n & Wet & 0.000000 & 1.000000 & 0.000000 & 0.000000 & 0.000000 & 0.700000 & 0.000000 & 0.300000 \\\\\n\\end{tabular}\n'